In [10]:
import numpy as np
import pandas as pd
import datetime
import time
import numba
import os,sys
import gc
import math

def LoadData(InputDir):
    """"""
    ## load raw data
    data = {
        'tra': pd.read_csv('%s/air_visit_data.csv' % InputDir, parse_dates= ['visit_date']),
        'as': pd.read_csv('%s/air_store_info.csv' % InputDir),
        'hs': pd.read_csv('%s/hpg_store_info.csv' % InputDir),
        'ar': pd.read_csv('%s/air_reserve.csv' % InputDir, parse_dates= ['visit_datetime', 'reserve_datetime']),
        'hr': pd.read_csv('%s/hpg_reserve.csv' % InputDir, parse_dates= ['visit_datetime', 'reserve_datetime']),
        'id': pd.read_csv('%s/store_id_relation.csv' % InputDir),
        'tes': pd.read_csv('%s/sample_submission.csv' % InputDir),
        'hol': pd.read_csv('%s/date_info.csv' % InputDir, parse_dates=['calendar_date']).rename(columns={'calendar_date': 'visit_date'})
    }
    return data

@numba.jit
def ApplyDayoff(VisitCols, ReserveCols):
    """"""
    n = len(VisitCols)
    result = np.zeros((n, 1), dtype= 'int8')
    for i in range(n):
        d = (VisitCols[i]- ReserveCols[i]).days
        if(d > 0):
            result[i] = d
    return result

reserve2id = {'ar': 'air', 'hr': 'hpg'}
reserve2store = {'ar': 'as', 'hr': 'hs'}# load data set
InputDir = '../../data/raw'
DataSet = LoadData(InputDir)
#### 
# date related features
print('\n============')
for mod in ['tra', 'tes']:
    start0 = time.time()
    if (mod == 'tes'):
        DataSet[mod]['visit_date'] = DataSet[mod]['id'].map(lambda x: str(x).split('_')[2])
        DataSet[mod]['air_store_id'] = DataSet[mod]['id'].map(lambda x: '_'.join(x.split('_')[:2]))
        DataSet[mod]['visit_date'] = pd.to_datetime(DataSet[mod]['visit_date'])
    DataSet[mod]['dow'] = DataSet[mod]['visit_date'].dt.dayofweek
    DataSet[mod]['year'] = DataSet[mod]['visit_date'].dt.year
    DataSet[mod]['month'] = DataSet[mod]['visit_date'].dt.month
    DataSet[mod]['visit_date'] = DataSet[mod]['visit_date'].dt.date
    end0 = time.time()
    print('%s data: unique stores %s, total %s, time elased %.2fs.' %
            (mod, len(DataSet[mod]['air_store_id'].unique()), len(DataSet[mod]['air_store_id']), (end0 - start0)))
print('============= process date related done.\n')
######## store data
# add city feature
for mod in ['ar', 'hr']:
    DataSet[reserve2store[mod]]['%s_city' % reserve2id[mod]] = DataSet[reserve2store[mod]]['%s_area_name' % reserve2id[mod]].str[:5]
print('add city feature done.')
# # divide area/genre into ten pieces
# for mod in ['as']:
#     DataSet[mod]['air_genre_name'] = DataSet[mod]['air_genre_name'].map(lambda x: str(str(x).replace('/',' ')))
#     DataSet[mod]['air_area_name'] = DataSet[mod]['air_area_name'].map(lambda x: str(str(x).replace('-',' ')))
#     for i in range(3):
#         DataSet[mod]['air_genre_name'+str(i)] = DataSet[mod]['air_genre_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else np.nan)
#         DataSet[mod]['air_area_name'+str(i)] = DataSet[mod]['air_area_name'].map(lambda x: str(str(x).split(' ')[i]) if len(str(x).split(' '))>i else np.nan)
# print('dividing area/genre into ten pieces done.')
# area (store)count
for mod in ['ar', 'hr']:
    rec = []
    groupped = DataSet[reserve2store[mod]].groupby(['%s_area_name' % reserve2id[mod]])
    for g in groupped.groups:
        ac = {}
        ac['%s_area_name' % reserve2id[mod]] = g
        ac['%s_area_store_count' % reserve2id[mod]] = len(groupped.get_group(g)['%s_store_id' % reserve2id[mod]].unique())
        #ac['%s_area_store_ratio' % reserve2id[mod]] = ac['%s_area_store_count' % reserve2id[mod]]/len(DataSet[reserve2store[mod]])
        rec.append(ac)
    tmpdf = pd.DataFrame(data= rec, index= range(len(rec)))
    DataSet[reserve2store[mod]] = DataSet[reserve2store[mod]].merge(tmpdf, how= 'left', on= ['%s_area_name' % reserve2id[mod]])
# genre (store)count
for mod in ['ar', 'hr']:
    rec = []
    groupped = DataSet[reserve2store[mod]].groupby(['%s_genre_name' % reserve2id[mod]])
    for g in groupped.groups:
        ac = {}
        ac['%s_genre_name' % reserve2id[mod]] = g
        ac['%s_genre_store_count' % reserve2id[mod]] = len(groupped.get_group(g)['%s_store_id' % reserve2id[mod]].unique())
        rec.append(ac)
    tmpdf = pd.DataFrame(data= rec, index= range(len(rec)))
    DataSet[reserve2store[mod]] = DataSet[reserve2store[mod]].merge(tmpdf, how= 'left', on= ['%s_genre_name' % reserve2id[mod]])
#  area_genre (store) count 
for mod in ['ar', 'hr']:
    rec = []
    groupby_keys = ['%s_area_name' % reserve2id[mod], '%s_genre_name' % reserve2id[mod]]
    groupped = DataSet[reserve2store[mod]].groupby(groupby_keys)
    for g in groupped.groups:
        ac = {}
        ac['%s_area_name' % reserve2id[mod]] = g[0]
        ac['%s_genre_name' % reserve2id[mod]] = g[1]
        ac['%s_area_genre_store_count' % reserve2id[mod]] = len(groupped.get_group(g)['%s_store_id' % reserve2id[mod]].unique())
        rec.append(ac)
    tmpdf = pd.DataFrame(data= rec, index= range(len(rec)))
    DataSet[reserve2store[mod]] = DataSet[reserve2store[mod]].merge(tmpdf, how= 'left', on= groupby_keys)
# city (store)count
for mod in ['ar', 'hr']:
    rec = []
    groupped = DataSet[reserve2store[mod]].groupby(['%s_city' % reserve2id[mod]])
    for g in groupped.groups:
        ac = {}
        ac['%s_city' % reserve2id[mod]] = g
        ac['%s_city_store_count' % reserve2id[mod]] = len(groupped.get_group(g)['%s_store_id' % reserve2id[mod]].unique())
        #ac['%s_area_store_ratio' % reserve2id[mod]] = ac['%s_area_store_count' % reserve2id[mod]]/len(DataSet[reserve2store[mod]])
        rec.append(ac)
    tmpdf = pd.DataFrame(data= rec, index= range(len(rec)))
    DataSet[reserve2store[mod]] = DataSet[reserve2store[mod]].merge(tmpdf, how= 'left', on= ['%s_city' % reserve2id[mod]])
#  city_genre (store) count 
for mod in ['ar', 'hr']:
    rec = []
    groupby_keys = ['%s_city' % reserve2id[mod], '%s_genre_name' % reserve2id[mod]]
    groupped = DataSet[reserve2store[mod]].groupby(groupby_keys)
    for g in groupped.groups:
        ac = {}
        ac['%s_city' % reserve2id[mod]] = g[0]
        ac['%s_genre_name' % reserve2id[mod]] = g[1]
        ac['%s_city_genre_store_count' % reserve2id[mod]] = len(groupped.get_group(g)['%s_store_id' % reserve2id[mod]].unique())
        rec.append(ac)
    tmpdf = pd.DataFrame(data= rec, index= range(len(rec)))
    DataSet[reserve2store[mod]] = DataSet[reserve2store[mod]].merge(tmpdf, how= 'left', on= groupby_keys)
print(' ================ add count features done.\n')
######### holiday data
data = DataSet['hol']
### add holiday days
data['visit_date'] = data['visit_date'].dt.date
data = data.sort_values(by= 'visit_date')
def TagHoliday(df):
    ''''''
    n = len(df)
    result = ['' for x in range(n)]
    for i in range(n):
        if(i == 0):
            result[i] = 'hid_%s' % 0
        elif((df[i] - df[i-1]).days == 1):
            result[i] = result[i - 1]
        else:
            result[i] = 'hid_%s' % (int(result[i - 1].split('_')[1]) + 1)
    return result
holidays = data[data['holiday_flg'] == 1][['visit_date']]
holidays['hol_l0'] = TagHoliday(holidays['visit_date'].values)
groupped = holidays.groupby(['hol_l0'])
recs = []
for g in groupped.groups:
    hol_days = {}
    hol_days['hol_l0'] = g
    hol_days['hol_days'] = len(groupped.get_group(g))
    recs.append(hol_days)
tmpdf = pd.DataFrame(data= recs, index= range(len(recs)))
holidays = holidays.merge(tmpdf, how= 'left', on= 'hol_l0')
data = data.merge(holidays, how= 'left', on= 'visit_date')
data.drop(['hol_l0'], axis= 1, inplace= True)
data['hol_days'].fillna(0, inplace= True)
print('add holiday type done.')
### reset holiday
wkend_holidays = data.apply((lambda x:(x.day_of_week=='Sunday' or x.day_of_week=='Saturday') and x.holiday_flg==1), axis=1)
data['is_weekends'] = (data['day_of_week'] == 'Sunday') | (data['day_of_week'] == 'Saturday')
data.loc[wkend_holidays, 'holiday_flg'] = 0
DataSet['hol'] = data
print('========== reset holiday done.\n')
######## join 
# join holiday data
for mod in ['tra', 'tes']:
    data = DataSet[mod]
    data = data.merge(DataSet['hol'], how='left', on=['visit_date'])
    data.drop(['day_of_week', 'year'], axis=1, inplace=True)
    DataSet[mod] = data
# join store data
for mod in ['tra', 'tes']:
    data = DataSet[mod]
    for rtype in ['ar', 'hr']: 
        if((rtype == 'hr') & (('%s_store_id' % reserve2id[rtype]) not in data.columns)):
            data = data.merge(DataSet['id'], how= 'left', on= ['air_store_id'])
        data = data.merge(DataSet[reserve2store[rtype]], how= 'left', on= ['%s_store_id' % reserve2id[rtype]])
    DataSet[mod] = data
print('================ join holiday, store data done.')
######### reservation data
for mod in ['hr', 'ar']:
    start1 = time.time()
    DataSet[mod]['visit_date'] = DataSet[mod]['visit_datetime'].dt.date
    DataSet[mod]['reserve_date'] = DataSet[mod]['reserve_datetime'].dt.date
    DataSet[mod].drop(['reserve_datetime', 'visit_datetime'], axis= 1, inplace= True)
    tmpdf = pd.DataFrame(data=ApplyDayoff(DataSet[mod]['visit_date'].values, DataSet[mod]['reserve_date'].values),index=DataSet[mod].index, columns=['reserve_date_diff'])
    tmpdf = pd.concat([DataSet[mod], tmpdf], axis=1)
    tmpdf = tmpdf.groupby(['%s_store_id' % reserve2id[mod], 'visit_date'], as_index=False).agg({'reserve_visitors': sum, 'reserve_date_diff': ['mean', 'median']})
    tmpdf.columns = ['%s_store_id' % reserve2id[mod], 
                   'visit_date', 
                   '%s_reserved_visitors' % reserve2id[mod], 
                   '%s_reserved_dayoff_mean' % reserve2id[mod], 
                   '%s_reserved_dayoff_median' % reserve2id[mod]
                  ]
    end1 = time.time()
    DataSet[mod] = tmpdf
    DataSet[mod]['%s_reserved_visitors' % reserve2id[mod]] = np.log1p(DataSet[mod]['%s_reserved_visitors' % reserve2id[mod]])
print(' process reservation data done.\n')
# join reservation data
for mod in ['tra', 'tes']:
    data = DataSet[mod]
    for rtype in ['ar', 'hr']: 
        if((rtype == 'hr') & (('%s_store_id' % reserve2id[rtype]) not in data.columns)):
            data = data.merge(DataSet['id'], how= 'left', on= ['air_store_id'])
        data = data.merge(DataSet[rtype], how= 'left', on= ['%s_store_id' % reserve2id[rtype], 'visit_date'])
        data['%s_store_id' % reserve2id[rtype]].fillna(0, inplace= True)
    DataSet[mod] = data
    DataSet[mod]['reserved_visitors_sum'] = DataSet[mod]['air_reserved_visitors'] + DataSet[mod]['hpg_reserved_visitors']
    DataSet[mod]['reserved_visitors_mean'] = (DataSet[mod]['air_reserved_visitors'] + DataSet[mod]['hpg_reserved_visitors'])/2
    DataSet[mod]['reserved_dayoff_mean'] = (DataSet[mod]['air_reserved_dayoff_mean'] + DataSet[mod]['hpg_reserved_dayoff_mean'])/2
    DataSet[mod]['reserved_dayoff_median'] = (DataSet[mod]['air_reserved_dayoff_median'] + DataSet[mod]['hpg_reserved_dayoff_median'])/2
print('============= join reservation data done.\n')
####### time series related
s = time.time()

# mix train with test
DataSet['tra']['is_train'] = 1
DataSet['tes']['is_train'] = 0
AllData = pd.concat([DataSet['tra'], DataSet['tes']], axis= 0, ignore_index= True)
# !!! dividing into two pieces since 2016/7/1 is a corner point, update time 2017/12/22 15:45
DataParts = {
    '0': AllData[AllData['visit_date'] < datetime.date(2016, 7, 1)],
    '1': AllData[AllData['visit_date'] >= datetime.date(2016, 7, 1)]
}
# manipulate with each of them
@numba.jit
def ApplySmoothWeek0(ColValues):
    ''''''
    n = len(ColValues)
    result = np.zeros((n, 1), dtype= 'float')
    alpha = 0.01
    for i in range(n):
        if(i < 5): # 5 weeks or 39 days
            result[i] = 0.0#math.log1p(ColValues[i])
        else:
            result[i] = alpha * math.log1p(ColValues[i - 5]) + (1.0 - alpha) * math.log1p(result[i - 5])
    return result

@numba.jit
def ApplySmoothDay0(ColValues):
    ''''''
    n = len(ColValues)
    result = np.zeros((n, 1), dtype= 'float')
    alpha = 0.8
    for i in range(n):
        if(i < 39): # 5 weeks or 39 days
            result[i] = 0.0#math.log1p(ColValues[i])
        else:
            result[i] = alpha * math.log1p(ColValues[i - 39]) + (1.0 - alpha) * math.log1p(result[i - 39])
    return result

@numba.jit
def ApplySmoothWeek1(ColValues):
    ''''''
    n = len(ColValues)
    result = np.zeros((n, 1), dtype= 'float')
    alpha = 0.01
    for i in range(n):
        if(i <= 5): # 5 weeks or 39 days
            result[i] = 0.0#math.log1p(ColValues[i])
        else:
            result[i] = alpha * (ColValues[i - 5] - ColValues[i - 6]) + (1.0 - alpha) * result[i - 5]
    return result

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

for pidx in DataParts.keys():
    ## exponential smoothing by weeks
#     groupped = DataParts[pidx].groupby(['air_store_id', 'dow'])
#     rmsle = 0
#     dfs = []
#     for g in groupped.groups: 
#         gdf = groupped.get_group(g).sort_values(by= ['visit_date'])
#         gdf['exp_smooth_week_0'] = ApplySmoothWeek0(gdf['visitors'].values)
#         #gdf['exp_smooth_week_1'] = ApplySmoothWeek1(gdf['exp_smooth_week_0'].values)
#         dfs.append(gdf)
#     tmpdf = pd.concat(dfs, axis= 0, ignore_index= True)
#     DataParts[pidx] = DataParts[pidx].merge(tmpdf[['air_store_id', 'visit_date', 'exp_smooth_week_0']], how= 'left', on= ['air_store_id', 'visit_date'])
#     ## exponential smoothing by days
#     groupped = DataParts[pidx].groupby(['air_store_id'])
#     rmsle = 0
#     dfs = []
#     for g in groupped.groups: 
#         gdf = groupped.get_group(g).sort_values(by= ['visit_date'])
#         gdf['exp_smooth_day_0'] = ApplySmoothDay0(gdf['visitors'].values)
#         #gdf['exp_smooth_week_1'] = ApplySmoothWeek1(gdf['exp_smooth_week_0'].values)
#         dfs.append(gdf)
#     tmpdf = pd.concat(dfs, axis= 0, ignore_index= True)
#     DataParts[pidx] = DataParts[pidx].merge(tmpdf[['air_store_id', 'visit_date', 'exp_smooth_day_0']], how= 'left', on= ['air_store_id', 'visit_date'])
#     print('part %s exponential smoothing done.' % pidx)
    ## rolling sum by days
    groupped = DataParts[pidx].groupby(['air_store_id'])
    visitor_ticks = [39, 46, 53, 60, 67, 74, 81]  # for days
    #visitor_ticks = [5, 10, 15, 20, 25]  # for weeks
    #reservation_ticks = [7, 14, 21, 28]
    print('total groups %s ' % len(groupped.groups))
    dfs = []
    for g in groupped.groups: 
        gdf = groupped.get_group(g).sort_values(by= ['visit_date'])
        for t in visitor_ticks:
            gdf['visitor_tick_sum_%s' % t] = np.log1p(gdf['visitors']).rolling(window= t).sum()
            gdf['visitor_tick_sum_%s' % t].fillna(0, inplace= True)
#             gdf['visitor_tick_std_%s' % t] = np.log1p(gdf['visitors']).rolling(window= t).std()
#             gdf['visitor_tick_std_%s' % t].fillna(0, inplace= True)
#         gdf['holiday_rolling_3'] = gdf['holiday_flg'].rolling(window= 3).sum()
#         gdf['holiday_rolling_3'].fillna(0, inplace= True)
        dfs.append(gdf)
    # concate
    tmpdf = pd.concat(dfs, axis= 0, ignore_index= True)
    join_cols = ['air_store_id', 'visit_date', 'holiday_rolling_3']
    for i in range(len(visitor_ticks)):
        if(i == 0):
            continue
        # rolling mean
        k_mean = 'visitor_rolling_%s_%s_mean' % (visitor_ticks[i], visitor_ticks[i - 1])
        tmpdf[k_mean] = (tmpdf['visitor_tick_sum_%s' % visitor_ticks[i]] - tmpdf['visitor_tick_sum_%s' % visitor_ticks[i - 1]]) / (visitor_ticks[i] - visitor_ticks[i - 1])
        tmpdf.loc[tmpdf[k_mean] < 0, k_mean] = 0  ## negative values exists, need to be set zero, updated 2016/12/22 20:30
        tmpdf[k_mean].fillna(0, inplace= True)
        join_cols.append(k_std)
#         # rolling std
#         k_std = 'visitor_rolling_%s_%s_std' % (visitor_ticks[i], visitor_ticks[i - 1])
#         tmpdf[k_std] = (tmpdf['visitor_tick_std_%s' % visitor_ticks[i]] - tmpdf['visitor_tick_std_%s' % visitor_ticks[i - 1]]) / (visitor_ticks[i] - visitor_ticks[i - 1])
#         tmpdf.loc[tmpdf[k_std] < 0, k_std] = 0  ## negative values exists, need to be set zero, updated 2016/12/22 20:30
#         tmpdf[k_std].fillna(0, inplace= True)
#         join_cols.append(k_mean)
    # merge
    tmpdf.drop(['visitor_tick_sum_%s' % col for col in visitor_ticks], axis= 1, inplace= True)
#     tmpdf.drop(['visitor_tick_std_%s' % col for col in visitor_ticks], axis= 1, inplace= True)
    DataParts[pidx] = DataParts[pidx].merge(tmpdf[join_cols], how= 'left', on= ['air_store_id', 'visit_date'])
    print('part %s rolling done.' % pidx)
# concat after all is done
AllData = pd.concat([DataParts['0'], DataParts['1']], axis= 0, ignore_index= True)
# restore
DataSet['tra'] = AllData[AllData['is_train'] == 1]
DataSet['tes'] = AllData[AllData['is_train'] == 0]
DataSet['tra'].drop(['is_train'], axis= 1, inplace= True)
DataSet['tes'].drop(['is_train'], axis= 1, inplace= True)
del AllData
print('add time series features done.')
#### add date_int
for mod in ['tra', 'tes']:
    DataSet[mod]['date_int'] = DataSet[mod]['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
    DataSet[mod]['date_int'] = DataSet[mod]['visit_date'].apply(lambda x: x.strftime('%Y%m%d')).astype(int)
print('add date int features done.')
#vdf = DataSet['tra'][DataSet['tra']['air_store_id'] == 'air_ba937bf13d40fb24']
#print(vdf[join_cols].head(200))
### add var_max_lat/var_max_long
for mod in ['tra', 'tes']:
    DataSet[mod]['lon_plus_lat_x'] = DataSet[mod]['longitude_x'] + DataSet[mod]['latitude_x'] 
#     DataSet[mod]['lon_plus_lat_y'] = DataSet[mod]['longitude_y'] + DataSet[mod]['latitude_y'] 
    DataSet[mod]['var_max_long_x'] = DataSet[mod]['longitude_x'].max() - DataSet[mod]['longitude_x']
    DataSet[mod]['var_max_lat_x'] = DataSet[mod]['latitude_x'].max() - DataSet[mod]['latitude_x']
#     DataSet[mod]['var_max_long_y'] = DataSet[mod]['longitude_y'].max() - DataSet[mod]['longitude_y']
#     DataSet[mod]['var_max_lat_y'] = DataSet[mod]['latitude_y'].max() - DataSet[mod]['latitude_y']
e = time.time()
print('time elapsed %ss' % ((e - s) * 60))
print(' ============= add time series related features done.\n')


tra data: unique stores 829, total 252108, time elased 0.85s.
tes data: unique stores 821, total 32019, time elased 0.13s.
============= process date related done.

add city feature done.
 ================ add count features done.

add holiday type done.
========== reset holiday done.

================ join holiday, store data done.
 process reservation data done.

============= join reservation data done.

total groups 316 
part 0 rolling done.
total groups 829 
part 1 rolling done.


/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:341: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


add time series features done.
add date int features done.
time elapsed 1517.8231859207153s
 ============= add time series related features done.



In [11]:
### fill nulls
from sklearn import *
cate_feats = ['genre_name', 'area_name', 'city']
cate_cols = ['%s_%s' % (m, cf) for m in ['air', 'hpg'] for cf in cate_feats]
for mod in ['tra', 'tes']:
    for col in DataSet[mod].columns:
        if(col in cate_cols):
            DataSet[mod][col].fillna('unknown', inplace= True)
        else:
            DataSet[mod][col].fillna(0, inplace= True)
print('Categorical features ', cate_cols)

Categorical features  ['air_genre_name', 'air_area_name', 'air_city', 'hpg_genre_name', 'hpg_area_name', 'hpg_city']


In [12]:
#### Label encoding for categorial features
TrainData = DataSet['tra']
TestData = DataSet['tes']
for col in cate_cols:
    lbl = preprocessing.LabelEncoder()
    TrainData[col] = lbl.fit_transform(TrainData[col])
    TestData[col] = lbl.transform(TestData[col])

In [13]:
import sys,os

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5
## !!! add corner tag since 2016/7/1 is a corner point, update time 2017/12/22 15:45
TrainData['is_up_corner'] = TrainData['visit_date'] < datetime.date(2016, 7, 1)
TestData['is_up_corner'] = TestData['visit_date'] < datetime.date(2016, 7, 1)
# split TrainData into train and holdout with random strategy
np.random.seed(2017)
msk = np.random.rand(len(TrainData)) < 0.1
holdout = TrainData[msk]
train = TrainData[~msk]
test = TestData
# Set up folds
K = 5
kf = model_selection.KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(1)
OutputDir = '../../data/l0'
if(os.path.exists('%s/kfold' % OutputDir) == False):
    os.makedirs('%s/kfold' % OutputDir)
for i, (train_index, test_index) in enumerate(kf.split(train)):
    FoldTrain, FoldValid = train.iloc[train_index].copy(), train.iloc[test_index].copy()
    FoldHoldout = holdout.copy()
    FoldTest = test.copy()
    FoldTrain['visitors'] = np.log1p(FoldTrain['visitors'])
    FoldValid['visitors'] = np.log1p(FoldValid['visitors'])
    FoldHoldout['visitors'] = np.log1p(FoldHoldout['visitors'])
    FoldTest['visitors'] = np.log1p(FoldTest['visitors'])
    #### dependent features which is extreemly subtle to data-leak
    # percentiles features
    tickles = ['mean', 'median', 'max', 'min', 'count']
    for feat in ['air_store_id']:
        gkeys = [feat, 'dow', 'is_up_corner']
        gprefix = '_'.join(gkeys)
        TmpDOW = FoldTrain.groupby(gkeys, as_index= False).agg({'visitors': tickles})    
        tmpcols = gkeys.copy()
        tmpcols.extend(['%s_%s' % (gprefix, m) for m in tickles])
        TmpDOW.columns = tmpcols
        #FoldTrain = FoldTrain.merge(TmpDOW, how= 'left', on=gkeys) #### data-leak, prone to be overfitted
        FoldValid = FoldValid.merge(TmpDOW, how= 'left', on=gkeys)
        FoldHoldout = FoldHoldout.merge(TmpDOW, how= 'left', on=gkeys)
        FoldTest = FoldTest.merge(TmpDOW, how= 'left', on=gkeys)
        FoldValid.fillna(0, inplace= True)
        FoldHoldout.fillna(0, inplace= True)
        FoldTest.fillna(0, inplace= True)
    for feat in ['air_city']:
        gkeys = [feat, 'air_genre_name', 'is_up_corner']
        gprefix = '_'.join(gkeys)
        TmpDOW = FoldTrain.groupby(gkeys, as_index= False).agg({'visitors': tickles})    
        tmpcols = gkeys.copy()
        tmpcols.extend(['%s_%s' % (gprefix, m) for m in tickles])
        TmpDOW.columns = tmpcols
        #FoldTrain = FoldTrain.merge(TmpDOW, how= 'left', on=gkeys) #### data-leak, prone to be overfitted
        FoldValid = FoldValid.merge(TmpDOW, how= 'left', on=gkeys)
        FoldHoldout = FoldHoldout.merge(TmpDOW, how= 'left', on=gkeys)
        FoldTest = FoldTest.merge(TmpDOW, how= 'left', on=gkeys)
        FoldValid.fillna(0, inplace= True)
        FoldHoldout.fillna(0, inplace= True)
        FoldTest.fillna(0, inplace= True)
    FoldOutputDir = '%s/kfold/%s' % (OutputDir, i)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    #FoldTrain.to_csv('%s/train.csv' % FoldOutputDir)
    FoldValid.to_csv('%s/valid.csv' % FoldOutputDir, index= False)
    FoldHoldout.to_csv('%s/holdout.csv' % FoldOutputDir, index= False)
    FoldTest.to_csv('%s/test.csv' % FoldOutputDir, index= False)
    
    print('Fold %s done.' % i)

Fold 0 done.
Fold 1 done.
Fold 2 done.
Fold 3 done.
Fold 4 done.
